In [1]:
# Time-Voting Post-Processing Evaluation
import os
import sys
import numpy as np
import pandas as pd
from collections import deque
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, f1_score

# Configuration
BASE_DIR = Path('../Results/Experiment_Result/Model_Preds/finetune_6class_other_existing_timevoting')
SUBJECT_IDS = ['201','202','203','205','206','207','208','209','210']
WINDOW_SIZES = [5, 10, 50, 100, 200, 300]
CLASS_NAMES = ["Other","Shower","Tooth_brushing","Vacuum_Cleaner","Washing_hands","Wiping"]
FINETUNE_SUBFOLDER = 'finetuned_full_420'
OUTPUT_CSV = BASE_DIR / 'time_voting_metrics_per_subject.csv'

if __name__ == '__main__':
    # Initialize results containers
    ba_results = {sid: {} for sid in SUBJECT_IDS}
    f1_results = {sid: {} for sid in SUBJECT_IDS}

    # Iterate subjects
    for sid in SUBJECT_IDS:
        pred_dir = BASE_DIR / sid / FINETUNE_SUBFOLDER
        if not pred_dir.is_dir():
            continue
        # Load predictions
        csvs = list(pred_dir.glob('*.csv'))
        if not csvs:
            continue
        df = pd.read_csv(csvs[0])
        y_true = df['y_true'].values
        probs = df[CLASS_NAMES].values

        # Time voting for each window size
        for w in WINDOW_SIZES:
            buf = deque()
            cum = np.zeros(len(CLASS_NAMES))
            y_pred_tv = []
            for p in probs:
                buf.append(p)
                cum += p
                if len(buf) > w:
                    cum -= buf.popleft()
                y_pred_tv.append(CLASS_NAMES[int(np.argmax(cum))])
            ba_results[sid][w] = balanced_accuracy_score(y_true, y_pred_tv)
            f1_results[sid][w] = f1_score(y_true, y_pred_tv, average='macro')

    # Convert to DataFrames
    ba_df = pd.DataFrame.from_dict(ba_results, orient='index', columns=WINDOW_SIZES)
    f1_df = pd.DataFrame.from_dict(f1_results, orient='index', columns=WINDOW_SIZES)
    ba_df.index.name = 'Subject'
    f1_df.index.name = 'Subject'

    # Scale and round
    ba_df = (ba_df * 100).round(2)
    f1_df = (f1_df * 100).round(2)

    # Add mean row
    ba_df.loc['Mean'] = ba_df.mean()
    f1_df.loc['Mean'] = f1_df.mean()

    # Combine with MultiIndex columns
    combined = pd.concat(
        {'Balanced_Accuracy': ba_df, 'Macro_F1': f1_df},
        axis=1
    )

    # Save results
    combined.to_csv(OUTPUT_CSV)
    print(f"Saved combined metrics to {OUTPUT_CSV}")

C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("

Saved combined metrics to ..\Results\Experiment_Result\Model_Preds\finetune_6class_other_existing_timevoting\time_voting_metrics_per_subject.csv
